In [27]:
import pymssql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
from statsmodels.tsa.stattools import adfuller, kpss
import itertools
import statsmodels.tsa.api as tsa

def DB_connection(tank_seq, low_error):
    conn = pymssql.connect(server='sql16ssd-014.localnet.kr', user='i2on11_admin', password='root0826', database='i2on11_admin')
    cursor = conn.cursor()

    conn_sql = 'SELECT signal_time, tank_remain_volume FROM gas_tank_volume_history where tank_seq = ' + tank_seq + 'ORDER BY signal_time DESC'

    # 쿼리 실행
    cursor.execute(conn_sql)

    # 결과 가져오기
    rows = cursor.fetchall()

    # 연결 닫기
    conn.close()
    d_h = pd.DataFrame(rows, columns=['datetime', 'history'])
    d_h['datetime'] = pd.to_datetime(d_h['datetime'])
    d_h.set_index('datetime', inplace=True)
    d_h.sort_index(ascending=True, inplace=True)
    d_h.drop(d_h[d_h['history'] <= 3].index, inplace=True)

    diff_d_h = d_h.copy()
    diff_d_h['history'] = diff_d_h['history'].diff()
    diff_d_h = diff_d_h.dropna()
    recent_datetime = diff_d_h['history'].index[-1] if (diff_d_h['history'] > 15).any() else diff_d_h.index.min()
    diff_d_h[diff_d_h >15] = 0
#     d_h = d_h[d_h.index>=recent_datetime]
#     diff_d_h = d_h.copy()
#     diff_d_h['history'] = diff_d_h['history'].diff()
#     diff_d_h = diff_d_h.dropna()
    
    diff_2d_h = diff_d_h.copy()
    diff_2d_h['history'] = diff_2d_h['history'].diff()
    diff_2d_h = diff_2d_h.dropna()

    diff_3d_h = diff_2d_h.copy()
    diff_3d_h['history'] = diff_3d_h['history'].diff()
    diff_3d_h = diff_3d_h.dropna()
    return d_h, diff_d_h, diff_2d_h, diff_3d_h

In [28]:
def adf_kpss_testing(history_diff):
    found_index = None
    
    for history_index, history in enumerate(history_diff):    
        try:
            # Perform Augmented Dickey-Fuller (ADF) test
            adf_result = adfuller(history)
            adf_statistic = adf_result[0]
            adf_pvalue = adf_result[1]
            adf_critical_values = adf_result[4]
            adf_test = (adf_pvalue < 0.05)
            adf_st_test = (adf_statistic >= -10)
            print('adf p-value : ', adf_pvalue, adf_test)
            print('adf statistic : ', adf_statistic, adf_st_test)
            # Perform Kwiatkowski-Phillips-Schmidt-Shin (KPSS) test
        except:
            adf_test = False
        try:
            kpss_result = kpss(history)
            kpss_statistic = kpss_result[0]
            kpss_pvalue = kpss_result[1]
            kpss_critical_values = kpss_result[3]
            kpss_test = (kpss_pvalue < 0.05)
        except:
            kpss_test= False
            print("kpss_test Warning")
            continue
        print(history_index,' kpss p-value : ', kpss_pvalue, kpss_test)
        acf = np.correlate(history, history, mode='full')[-len(history):]
        acf_test = (acf[1]>=0 and acf[2]>=0 and acf[1]>=acf[2]) or (acf[1]<0 and acf[2]>=0)
        print(acf_test)
        
        if history_index == 0:
            continue
        if adf_test and acf_test and not kpss_test:
            found_index = history_index - 1
            break
        
    if found_index is not None:
        print("Index:", found_index)
    else:
        found_index = 0
        print("Index not found.")
    return found_index

In [29]:
def none_differencing_func(pred_h, pred_df, datetime, prevalue):
    for i in range(0, len(pred_df['mean'])):
        datetime = pred_df.index[i]
        prevalue = pred_df['mean'][i]
        pred_h[str(datetime)] = str(prevalue)
    return pred_h

In [30]:
def first_differencing_func(pred_h, pred_df, datetime, prevalue):
    for i in range(0, len(pred_df['mean'])):
        datetime = pred_df.index[i]  # access datetime using index
        prevalue = round(prevalue + pred_df['mean'][i],3)
        pred_h[str(datetime)] = str(prevalue)
    return pred_h

In [31]:
def second_differencing_func(pred_h, pred_df, prevalue, datetime, diffmaxindex, differing):
    for i in range(0, len(pred_df['mean'])):
        differing = differing + pred_df['mean'][i]
        datetime = pred_df.index[i]  # access datetime using index
        prevalue = round(prevalue + differing,3)
        pred_h[str(datetime)] = str(prevalue)
    return pred_h

In [32]:
def ARIMA_prediction(found_index, history_diff):
    rcParams['figure.figsize'] = (15, 20)
    
    d_h = history_diff[0]
    diff_d_h = history_diff[1]
#     plt.figure(figsize=(10, 5))
#     plt.plot(d_h)
    
    y=history_diff[found_index].resample('1D').mean()
    y.isnull().sum()
    y1 = y.fillna(method='ffill')
    p=d=q=range(0,2)
    pdq = list(itertools.product(p,d,q))
    seasonal_pdq = [(x[0],x[1],x[2]) for x in  list(itertools.product(p,d,q))]

    for param in pdq:
        try:
            mod = tsa.statespace.SARIMAX(y1, order=param, enforce_stationarity=False, enforce_invertibility=False)
            results = mod.fit()
            param_list.append(param)
            results_AIC_list.append(results.aic)
        except:
            continue
    # print(param)
    mod = tsa.statespace.SARIMAX(y1, order=param,
                                enforce_stationarity=True, enforce_invertibility=False)
    results=mod.fit()
    max_date = history_diff[found_index].index.max()
    new_date = max_date + pd.DateOffset(days=14)
#     print(new_date)
    pred = results.get_prediction(start=pd.to_datetime(max_date),
                                 end=pd.to_datetime(new_date), dynamic=True)
    pred_df = pred.summary_frame()
    pred_df.rename_axis('datetime', inplace=True)
    # Extract the hours, minutes, and seconds from max_date
    max_hours = max_date.hour
    max_minutes = max_date.minute
    max_seconds = max_date.second

    # Set the hours, minutes, and seconds of the datetime column in pred_df
    pred_df.index = pred_df.index.map(lambda timestamp: timestamp.replace(
        hour=max_hours, minute=max_minutes, second=max_seconds
    ))
    prevalue = d_h.loc[d_h.index.max()]
    datetime_max = d_h.index.max()
    diffmaxindex = diff_d_h.index.max()
    differing = diff_d_h.loc[diffmaxindex]
    pred_h = {str(datetime_max) : str(prevalue)}
#     print(found_index)
    if found_index == 1:
        pred_h = first_differencing_func(pred_h, pred_df, datetime_max, prevalue)
#     elif 
#         pred_h = none_differencing_func(pred_h, pred_df, datetime_max, prevalue)
    elif found_index == 2:
        pred_h = second_differencing_func(pred_h, pred_df, prevalue, datetime_max, diffmaxindex, differing)
    else :
        pred_h = none_differencing_func(pred_h, pred_df, datetime_max, prevalue)
    return pred_h

In [33]:
def prediction_by_tank_seq(tank_seq):
    d_h, diff_d_h, diff_2d_h, diff_3d_h = DB_connection(tank_seq, True)
    d_h = DB_connection(tank_seq, False)[0]
    history_diff = [d_h['history'], diff_d_h['history'], diff_2d_h['history'], diff_3d_h['history']]
    found_index = adf_kpss_testing(history_diff)
    pred_h = ARIMA_prediction(found_index, history_diff)
    return pred_h

print(prediction_by_tank_seq('302'))

adf p-value :  9.373961375024746e-05 True
adf statistic :  -4.674999480932716 True
0  kpss p-value :  0.1 False
True
adf p-value :  4.624933337203695e-06 True
adf statistic :  -5.335313162113159 True
1  kpss p-value :  0.01 True
False
adf p-value :  5.3735929857669583e-20 True
adf statistic :  -11.040755205842085 False
2  kpss p-value :  0.1 False
True
Index: 1


C:\Users\hdenergy\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:2022: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
C:\Users\hdenergy\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:2018: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(
C:\Users\hdenergy\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:2022: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


{'2023-05-25 16:37:38': '61', '2023-05-26 16:37:38': '60.105', '2023-05-27 16:37:38': '59.447', '2023-05-28 16:37:38': '58.727', '2023-05-29 16:37:38': '58.023', '2023-05-30 16:37:38': '57.315', '2023-05-31 16:37:38': '56.608', '2023-06-01 16:37:38': '55.901', '2023-06-02 16:37:38': '55.194', '2023-06-03 16:37:38': '54.487', '2023-06-04 16:37:38': '53.78', '2023-06-05 16:37:38': '53.073', '2023-06-06 16:37:38': '52.366', '2023-06-07 16:37:38': '51.659', '2023-06-08 16:37:38': '50.952', '2023-06-09 16:37:38': '50.245'}


In [ ]:
from flask import Flask
from flask import request
from flask import jsonify
from datetime import datetime
app = Flask(__name__)

@app.route('/test', methods=['POST'])
def seaqq():
    data = request.form
    tank_seq = data['tank_seq']

    runningvalue = prediction_by_tank_seq(tank_seq)
    stringreturnvalue = str(runningvalue)
    print(stringreturnvalue)

    return jsonify({'result': runningvalue})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port="5000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.19:5000/ (Press CTRL+C to quit)
